In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path

In [ ]:
fp = "../data/olist_prepared/SP_lat_long.csv"
dfll = pd.read_csv(fp)

In [ ]:
dfll

In [ ]:
fp = "../data/olist_prepared/SP_connected_data_2017.csv"
dfconn = pd.read_csv(fp)

In [ ]:
dfconn = dfconn.rename(columns={"city": "name"})

In [ ]:
dfconn

In [ ]:
df_cities = pd.merge(dfconn, dfll, on="name")

In [ ]:
df_cities

# Data Errors in the customer state column
It looks like there are data errors in the customer state columns. These may be intentional or an artifact of the errors in data engineering. This can happen. Originally when I plotted the latitude and longitude of some cities labeled as in SP state in the data, I got values of latitude and longitude outside the Sao Paulo region. The range of latitude and longitude were determined from a google search and then these were applied to remove the errorneous points. There may still be some errors because these estimates for the range of latitude for Sao Paulo state may need tuning.

In [ ]:
df_cities.latitude.describe()

In [ ]:
df_cities.longitude.describe()

In [ ]:
lat_min = - 26
lat_max = - 20
valid_lat = (df_cities.latitude >= lat_min) & (df_cities.latitude <= lat_max)
df_cities = df_cities[valid_lat]
long_min = -56 
long_max = -44
valid_long = (df_cities.longitude >= long_min) & (df_cities.longitude <= long_max)
df_cities = df_cities[valid_long]
df_cities.shape


In [ ]:
gdf_iid = geopandas.GeoDataFrame(
    df_cities, geometry=geopandas.points_from_xy(df_cities.longitude, df_cities.latitude), crs="EPSG:4326"
)


In [ ]:
gdf_iid

In [ ]:
import geobr
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [ ]:
gdf_iid

In [ ]:
# Plot all Brazilian states
# State of Sao Paulo
states = state = geobr.read_state(code_state="SP", year=2018)

fig, ax = plt.subplots(figsize=(15, 15), dpi=300)

base = states.plot(facecolor="white", edgecolor="black", ax=ax)
gdf_iid.plot(ax=base, color="red")
for x, y, label in zip(gdf_iid.geometry.x, gdf_iid.geometry.y, gdf_iid['name']):
    ax.text(x, y, label, fontsize=10)
ax.set_title("Cities where Purchase Behavior is connected", fontsize=20)

ax.axis("off")